# Sequential Workflow with Custom Executors

## Overview

This notebook demonstrates mixing agents and **custom executors** in a sequential workflow. We'll create a pipeline where:
1. A content agent generates detailed text
2. A custom summarizer executor processes the conversation

This pattern shows how to create custom executors that work with the shared conversation context in sequential orchestration.

### Key Concepts:

1. **Mixing Participants**: Combine agents and custom executors
2. **Custom Executor Contract**: Accept `list[ChatMessage]`, emit updated list
3. **Conversation Processing**: Read, transform, and append to shared context
4. **Internal Adapters**: Automatic conversion between agent and executor formats

### Architecture:

```
User Prompt
    ↓
[input-conversation]
    ↓
Content Agent (generates paragraph)
    ↓
[to-conversation:content]
    ↓
Summarizer Executor (custom)
    ↓
[complete]
    ↓
Final Conversation with Summary
```

## Prerequisites

- Azure OpenAI configured with environment variables
- Azure CLI authentication: Run `az login`
- Agent Framework installed

## Setup and Imports

In [ ]:
import asyncio
from typing import Any

import os
from dotenv import load_dotenv
from agent_framework import (
    ChatMessage,
    Executor,
    Role,
    SequentialBuilder,
    WorkflowContext,
    handler,
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from typing_extensions import Never
# Load environment variables from .env file
load_dotenv('../../.env')


## Create Custom Summarizer Executor

### Custom Executor Requirements:

1. **Handler signature**: Accept `list[ChatMessage]` and `WorkflowContext[list[ChatMessage]]`
2. **Processing**: Read the conversation, analyze it
3. **Output**: Emit updated conversation via `ctx.yield_output()`

### Our Summarizer:
- Counts user and assistant messages
- Appends a summary message to the conversation

In [ ]:
class Summarizer(Executor):
    """Simple summarizer: consumes full conversation and appends an assistant summary."""

    @handler
    async def summarize(self, conversation: list[ChatMessage], ctx: WorkflowContext[Never, list[ChatMessage]]) -> None:
        users = sum(1 for m in conversation if m.role == Role.USER)
        assistants = sum(1 for m in conversation if m.role == Role.ASSISTANT)
        summary = ChatMessage(role=Role.ASSISTANT, text=f"Summary -> users:{users} assistants:{assistants}")
        final_conversation = list(conversation) + [summary]
        await ctx.yield_output(final_conversation)

## Build and Execute Sequential Workflow

### Participant Order:
1. **Content agent**: Generates detailed response
2. **Summarizer executor**: Processes and summarizes conversation

In [ ]:
async def run_sequential_custom_executors() -> None:
    # Create content agent
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    chat_client = AzureOpenAIChatClient(
        deployment_name=deployment_name,
        endpoint=endpoint,
        credential=AzureCliCredential()
    )
    content = chat_client.create_agent(
        instructions="Produce a concise paragraph answering the user's request.",
        name="content",
    )

    # Build sequential workflow: content -> summarizer
    summarizer = Summarizer(id="summarizer")
    workflow = SequentialBuilder().participants([content, summarizer]).build()

    # Run and print final conversation
    events = await workflow.run("Explain the benefits of budget eBikes for commuters.")
    outputs = events.get_outputs()

    if outputs:
        print("===== Final Conversation =====")
        messages: list[ChatMessage] | Any = outputs[0]
        for i, msg in enumerate(messages, start=1):
            name = msg.author_name or ("assistant" if msg.role == Role.ASSISTANT else "user")
            print(f"{'-' * 60}\n{i:02d} [{name}]\n{msg.text}")

## Run the Workflow

In [ ]:
await run_sequential_custom_executors()

## Expected Output

```
===== Final Conversation =====
------------------------------------------------------------
01 [user]
Explain the benefits of budget eBikes for commuters.
------------------------------------------------------------
02 [content]
Budget eBikes offer commuters an affordable, eco-friendly alternative to cars and public transport.
Their electric assistance reduces physical strain and allows riders to cover longer distances quickly,
minimizing travel time and fatigue. Budget models are low-cost to maintain and operate, making them accessible
for a wider range of people. Additionally, eBikes help reduce traffic congestion and carbon emissions,
supporting greener urban environments. Overall, budget eBikes provide cost-effective, efficient, and
sustainable transportation for daily commuting needs.
------------------------------------------------------------
03 [assistant]
Summary -> users:1 assistants:1
```

## Key Takeaways

### 1. Custom Executor Handler Contract
```python
@handler
async def handler_name(
    self,
    conversation: list[ChatMessage],
    ctx: WorkflowContext[Never, list[ChatMessage]]
) -> None:
    # Process conversation
    # Emit updated conversation
    await ctx.yield_output(updated_conversation)
```

### 2. Mixing Agents and Executors
- Agents and executors can be freely mixed in `.participants([...])`
- Internal adapters handle format conversions
- Shared conversation context flows through all

### 3. Conversation Processing Patterns

#### Counting/Analysis:
```python
users = sum(1 for m in conversation if m.role == Role.USER)
assistants = sum(1 for m in conversation if m.role == Role.ASSISTANT)
```

#### Content Extraction:
```python
last_assistant = next(
    (m for m in reversed(conversation) if m.role == Role.ASSISTANT),
    None
)
```

#### Transformation:
```python
processed = [transform_message(m) for m in conversation]
```

### 4. Appending to Conversation
```python
# Create new message
summary = ChatMessage(role=Role.ASSISTANT, text="...")

# Append to conversation
final_conversation = list(conversation) + [summary]

# Emit via context
await ctx.yield_output(final_conversation)
```

### 5. Use Cases for Custom Executors

#### Analytics Executor:
```python
class AnalyticsExecutor(Executor):
    @handler
    async def analyze(self, conversation, ctx):
        metrics = {
            "total_tokens": calculate_tokens(conversation),
            "sentiment": analyze_sentiment(conversation),
            "topics": extract_topics(conversation)
        }
        await ctx.yield_output(conversation)  # Pass through unchanged
        # Log metrics separately
```

#### Validation Executor:
```python
class ValidationExecutor(Executor):
    @handler
    async def validate(self, conversation, ctx):
        last_response = conversation[-1].text
        if not meets_criteria(last_response):
            error_msg = ChatMessage(
                role=Role.SYSTEM,
                text="Validation failed: regenerate response"
            )
            await ctx.yield_output(conversation + [error_msg])
        else:
            await ctx.yield_output(conversation)
```

#### Formatting Executor:
```python
class FormattingExecutor(Executor):
    @handler
    async def format(self, conversation, ctx):
        formatted = ChatMessage(
            role=Role.ASSISTANT,
            text=format_as_markdown(conversation)
        )
        await ctx.yield_output(conversation + [formatted])
```

### 6. Internal Adapter Nodes
Sequential workflows include:
- **`input-conversation`**: Normalizes input to list[ChatMessage]
- **`to-conversation:<participant>`**: Converts agent responses
- **`complete`**: Signals completion

These appear in event streams but can be safely ignored when focusing on your agents/executors.

### 7. WorkflowContext Usage
- **`ctx.yield_output(data)`**: Emit final workflow output
- **`ctx.send_message(data)`**: Send to next executor (if in non-sequential workflow)
- **`ctx.add_event(event)`**: Emit custom events

### 8. Production Considerations
- Keep custom executors lightweight and focused
- Add error handling for malformed conversations
- Log executor processing for debugging
- Consider timeout handling for complex processing
- Test custom executors independently before integration